In [1]:
import socket
import struct
import csv
import numpy as np
import IPython.display as disp

print "modules loaded"

sdate = '20150417'
spath = '/home/duxbury/ipython/user/'+sdate+'/'
sconnect = spath + 'lda_scores.csv'
topct = 500

modules loaded


In [2]:
#!cat $sconnect

In [ ]:
#sev,tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt,lda_score
conns_list = []
with open(sconnect, 'r') as f:
    reader = csv.reader(f,delimiter=',') 
    reader.next();
    rowct = 1
    for row in reader:
        if int(row[0]) < 3: # 3 is the don't care
            # get src ip and dst ip
            sip = row[2]
            dip = row[3]
            # get hour and date 2014-07-08 10:10:40
            print row[1]
            hr = row[1].split(' ')[1].split(':')[0]
            dy = row[1].split(' ')[0].split('-')[2] 
            mm = row[1].split(' ')[1].split(':')[1]
            #TODO: using netflow_avro table, this query should change - no more minute(), hour()
            # also, there are more columns to return           
            conn = (sip,dip,dy,hr,mm)
            if conn not in conns_list:
                conns_list.append(conn)                    

            if rowct == topct:
                break
            rowct = rowct + 1

for conn in conns_list:
    sip = conn[0]
    dip = conn[1]
    dy = conn[2]
    hr = conn[3]
    mm = conn[4]
    
    hivestr = (" \"set hive.cli.print.header=true; SELECT treceived as tstart,sip as srcip," +
    "dip as dstip,sport as sport,dport as dport,proto as proto,flag as flags,stos as TOS," +
    "ibyt as bytes, ipkt as pkts,input as input, output as output from netflow" + 
    " WHERE ( (sip=\'" + sip + "\' AND dip=\'" + dip + "\') OR " +
    "(sip=\'" + dip + "\' AND dip=\'" + sip + "\') ) AND trday="+dy+" AND trhour="+hr +
    " AND trminute="+mm +" SORT BY tstart LIMIT 100 \"  > " + spath+ "edge-" + sip.replace(".","_") + "-" + 
    dip.replace(".","_") + "-" + hr + "-" + mm + "_tst.tsv")
    disp.clear_output()
    print 'processing line ',rowct
    print hivestr
    !hive -S -e $hivestr


processing line  500
 "set hive.cli.print.header=true; SELECT tstart as tstart,sip as srcip,dip as dstip,sport as sport,dport as dport,proto as proto,flag as flags,stos as TOS,ibyt as bytes, ipkt as pkts,input as input, output as output from netflow_poc WHERE ( (sip='222.92.56.26' AND dip='134.134.139.72') OR (sip='134.134.139.72' AND dip='222.92.56.26') ) AND day='17' AND hour='03' AND minute(tstart)=25 SORT BY tstart LIMIT 100 "  > /home/hadoop/ipython/user/20150417/edge-222_92_56_26-134_134_139_72-03-25_tst.tsv
15/08/10 12:47:08 WARN conf.HiveConf: DEPRECATED: Configuration property hive.metastore.local no longer has any effect. Make sure to provide a valid value for hive.metastore.uris if you are connecting to a remote metastore.


In [172]:
#sev,tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt,lda_score
srcdict = {}
rowct = 1
with open(sconnect, 'r') as f:
    reader = csv.reader(f,delimiter=',') 
    reader.next();
    rowct = 1
    for row in reader:
        if row[2] in srcdict:
            srcdict[row[2]] += 1
        else:
            srcdict[row[2]] = 1
        if row[3] in srcdict:
            srcdict[row[3]] += 1
        else:
            srcdict[row[3]] = 1
        rowct += 1
        if rowct == topct:
            break
print len(srcdict)

321


In [ ]:
ipct = 1
for ip in srcdict:
    rowct = 1
    if srcdict[ip] > 1:
        # get src ip and dst ip
        # iterate through the list again for any risk value > 3
        # build a dstdict of keys
        dstdict = {}
        with open(sconnect, 'r') as f:
            reader = csv.reader(f,delimiter=',') 
            reader.next();
            for row in reader:
                if ip == row[2]:
                    dstdict[row[3]]=row[3]
                if ip == row[3]:
                    dstdict[row[2]]=row[2]
                rowct += 1
                if rowct == topct:
                    break        
        # query hive with aggregated query 
        if len(dstdict.keys()) > 1:
            dstip = "'%s',"*len(dstdict.keys()) % tuple(dstdict.keys())
            hivestr = (" \"set hive.cli.print.header=true; SELECT sip as srcip," +
            "dip as dstip, MAX(ibyt) as maxbyte, AVG(ibyt) as avgbyte,  MAX(ipkt) as maxpkt, AVG(ipkt) as avgpkt " +
            " from netflow" +  
            " WHERE trday="+dy+" AND ( (sip=\'" + ip + "\' AND dip IN("+ dstip[:-1] + ")"
            " OR sip IN(" + dstip[:-1] + ") AND dip=\'" + ip + "\') ) " +
            " GROUP BY sip,dip \"  > " + spath + "chord-" + ip.replace(".","_") + ".tsv")
            disp.clear_output()
            print 'processing line ',ipct
            print hivestr
            !hive -S -e $hivestr
    if ipct == topct:
        break
    ipct += 1


processing line  1
 "set hive.cli.print.header=true; SELECT sip as srcip,dip as dstip, MAX(ibyt) as maxbyte, AVG(ibyt) as avgbyte,  MAX(ipkt) as maxpkt, AVG(ipkt) as avgpkt  from netflow_poc WHERE day='23' AND ( (sip='176.109.224.84' AND dip IN('134.134.137.73','192.55.54.38','134.134.139.74') OR sip IN('134.134.137.73','192.55.54.38','134.134.139.74') AND dip='176.109.224.84') )  GROUP BY sip,dip "  > /home/hadoop/ipython/user/20150423/chord-176_109_224_84.tsv
15/06/16 08:57:22 WARN conf.HiveConf: DEPRECATED: Configuration property hive.metastore.local no longer has any effect. Make sure to provide a valid value for hive.metastore.uris if you are connecting to a remote metastore.


In [ ]:
from os import walk

chordfiles = []
for (dirpath, dirnames, filenames) in walk(spath):
    for files in filenames:
        if files.startswith("chord-") and files.endswith(".tsv"):
            chordfiles.extend([files])
    break

In [ ]:
#ceate chord matrix
for chordfile in chordfiles:
    #chordfile = 'chord-10_0_0_9'
    ll = 15;
    #load csv into array
    #with open(spath+chordfile+'.tsv', 'rb') as f:
    with open(spath+chordfile, 'rb') as f:
        reader = csv.reader(f,delimiter='\t') 
        reader.next()
        carry = np.zeros([ll,3],np.int64)
        rowct = 0
        for row in reader:
            carry[rowct,0] = struct.unpack("!L", socket.inet_aton(row[0]))[0]    
            carry[rowct,1] = struct.unpack("!L", socket.inet_aton(row[1]))[0]    
            carry[rowct,2] = np.float32(row[3])
            rowct += 1
            if rowct == ll-1:
                break

    dim = np.zeros(2*ll,np.int64)
    dim = np.concatenate([carry[:,0],carry[:,1]],axis=0)
    dim = np.unique(dim)
    #print dim
    chordm = np.zeros([len(dim),len(dim)],np.int64)
    for j in carry:
        xcoord = np.where(dim == j[0]);
        ycoord = np.where(dim == j[1]);
        chordm[xcoord,ycoord] = j[2]
   
    dimstr = []
    for j in dim:
        dimstr.append(socket.inet_ntoa(struct.pack('!L', j)))
    headr = ",".join(dimstr)
    fmtr = r'%1d ' * len(dim)
    #with open(spath+chordfile+'.csv', 'wb') as f:
    with open(spath+chordfile.replace('.tsv','.csv'), 'wb') as f:
        riter = csv.writer(f,delimiter=',')
        # header line?
        for j in chordm:
            #outstr = '['+','.join(map(str, j))+'],'
            riter.writerow(j)
    #np.savetxt(g,chordm,delimiter=',',fmt=fmtr,header=headr)
print 'chord data loaded.'